In [2]:
import numpy as np 
import pandas as pd 


In [6]:
train = pd.read_csv('/Users/senlinlidewo/金融风控比赛/用户逾期/train.csv')
test = pd.read_csv('/Users/senlinlidewo/金融风控比赛/用户逾期/test.csv')

/Users/senlinlidewo/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [12]:
print(train.shape)
print(test.shape)

(285285, 627)
(189766, 626)


In [99]:
%%time
train.sample(5)
#随机选一些样本来查看

CPU times: user 11.8 ms, sys: 11.4 ms, total: 23.3 ms
Wall time: 25.2 ms


,CUST_ID,OPEN_ORG_NUM,IDF_TYP_CD,GENDER,bad_good,LAST_OPEN_TENURE_DAYS,G_OS_PRCP_SUM,OS_PRCP_SUM_THREE,OS_PRCP_SUM_SIX,guozhai_flag,...,L3_CHANNEL_TXN_DOUTTA_AVGCNT,L6_CHANNEL_TXN_STAIN_AVGAMT,L6_CHANNEL_TXN_SOUTTA_AVGAMT,L6_CHANNEL_TXN_DTAIN_AVGAMT,L6_CHANNEL_TXN_DOUTTA_AVGAMT,L6_CHANNEL_TXN_STAIN_AVGCNT,L6_CHANNEL_TXN_SOUTTA_AVGCNT,L6_CHANNEL_TXN_DTAIN_AVGCNT,L6_CHANNEL_TXN_DOUTTA_AVGCNT,gender
9014,1038807941,114,ZR01,2,0,0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,2
114963,1221668606,6600,ZR01,2,0,14,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,2
21522,1081619674,321,ZR01,2,0,0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,2
97226,1203768645,2008,ZR01,1,0,0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,1
6380,1028383696,506,ZR01,2,0,0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,2


In [28]:
missing_values_count = train.isnull().sum()
print(missing_values_count)

CUST_ID                         0
OPEN_ORG_NUM                    0
IDF_TYP_CD                      0
GENDER                          0
bad_good                        0
                               ..
L6_CHANNEL_TXN_DOUTTA_AVGAMT    0
L6_CHANNEL_TXN_STAIN_AVGCNT     0
L6_CHANNEL_TXN_SOUTTA_AVGCNT    0
L6_CHANNEL_TXN_DTAIN_AVGCNT     0
L6_CHANNEL_TXN_DOUTTA_AVGCNT    0
Length: 627, dtype: int64
0


In [104]:
total_missing=missing_values_count.sum()
print(total_missing)
#有多少缺失值

0


In [19]:
total_cells=np.product(train.shape)
total_cells
#有多少数据格子

178873695

In [101]:
total_missing/total_cells 
#缺失比率

0.0

In [103]:
#第二种缺失率求法
def missing_percentage(df):
    """This function takes a DataFrame(df) as input and returns two columns, total missing values and total missing values percentage"""
    total = df.isnull().sum().sort_values(ascending = False)
    percent = round(df.isnull().sum().sort_values(ascending = False)/len(df)*100,2)
    return pd.concat([total, percent], axis=1, keys=['Total','Percent'])
missing_percentage(train)

,Total,Percent
gender,0,0.0
L3_CHANNEL_CTR_FINANCIAL_MONTHAV,0,0.0
L6_CHANNEL_CTR_FUND_MONTHAVG_CNT,0,0.0
L6_CHANNEL_CTR_FUND_MONTHAVG_AMT,0,0.0
L3_CHANNEL_CTR_GLOAN_MONTHAVG_CN,0,0.0
...,...,...
L6_CHANNEL_OTRPOS_CREDIT_AVG_CNT,0,0.0
L6_CHANNEL_OTRPOS_CREDIT_AVG_AMT,0,0.0
L6_CHANNEL_OTRPOS_DEBIT_AVG_CNT,0,0.0
L6_CHANNEL_OTRPOS_DEBIT_AVG_AMT,0,0.0


In [ ]:
train['bad_good'].fillna(train['bad_good'].median(),inplace=True)
train['bad_good'].fillna('S',inplace=True)#最高特征值填充
#补充缺失值

In [105]:
#其他补充缺失值的方法
def percent_value_counts(df, feature):
    """This function takes in a dataframe and a column and finds the percentage of the value_counts"""
    percent = pd.DataFrame(round(df.loc[:,feature].value_counts(dropna=False, normalize=True)*100,2))
    ## creating a df with th
    total = pd.DataFrame(df.loc[:,feature].value_counts(dropna=False))
    ## concating percent and total dataframe

    total.columns = ["Total"]
    percent.columns = ['Percent']
    return pd.concat([total, percent], axis = 1)
percent_value_counts(train,'GENDER')


,Total,Percent
1,112409,39.40
2,99310,34.81
1,37305,13.08
2,35220,12.35
X,1041,0.36


In [127]:
#删除某行如果含有NA值
train.dropna(subset=['GENDER','bad_good'],how='any')

,CUST_ID,OPEN_ORG_NUM,IDF_TYP_CD,GENDER,bad_good,LAST_OPEN_TENURE_DAYS,G_OS_PRCP_SUM,OS_PRCP_SUM_THREE,OS_PRCP_SUM_SIX,guozhai_flag,...,L3_CHANNEL_TXN_DOUTTA_AVGCNT,L6_CHANNEL_TXN_STAIN_AVGAMT,L6_CHANNEL_TXN_SOUTTA_AVGAMT,L6_CHANNEL_TXN_DTAIN_AVGAMT,L6_CHANNEL_TXN_DOUTTA_AVGAMT,L6_CHANNEL_TXN_STAIN_AVGCNT,L6_CHANNEL_TXN_SOUTTA_AVGCNT,L6_CHANNEL_TXN_DTAIN_AVGCNT,L6_CHANNEL_TXN_DOUTTA_AVGCNT,gender
0,1000000024,1101,ZR01,2,0,0,204750.0,197750.00000,98875.00000,0,...,0,0,0,0,0,0,0,0,0,2
1,1000000515,602,ZR01,1,0,0,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,1
2,1000003231,602,ZR01,1,1,6,0.0,466666.66667,233333.33333,0,...,0,0,0,0,0,0,0,0,0,1
3,1000003736,602,ZR01,1,0,0,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,1
4,1000003852,602,ZR01,1,0,26,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285280,1500249699,2113,ZR01,1,0,0,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,1
285281,1500249715,5201,ZR01,1,0,0,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,1
285282,1500249833,120,ZR01,1,0,0,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,1
285283,1500249910,3006,ZR01,1,0,0,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,1


In [106]:
#查看缺失值所在记录
train[train.GENDER.isnull()]

,CUST_ID,OPEN_ORG_NUM,IDF_TYP_CD,GENDER,bad_good,LAST_OPEN_TENURE_DAYS,G_OS_PRCP_SUM,OS_PRCP_SUM_THREE,OS_PRCP_SUM_SIX,guozhai_flag,...,L3_CHANNEL_TXN_DOUTTA_AVGCNT,L6_CHANNEL_TXN_STAIN_AVGAMT,L6_CHANNEL_TXN_SOUTTA_AVGAMT,L6_CHANNEL_TXN_DTAIN_AVGAMT,L6_CHANNEL_TXN_DOUTTA_AVGAMT,L6_CHANNEL_TXN_STAIN_AVGCNT,L6_CHANNEL_TXN_SOUTTA_AVGCNT,L6_CHANNEL_TXN_DTAIN_AVGCNT,L6_CHANNEL_TXN_DOUTTA_AVGCNT,gender


In [ ]:
#更多操作：https://www.kaggle.com/alexisbcook/missing-values

In [43]:
train.info() #也可以用作查找缺失值

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285285 entries, 0 to 285284
Columns: 627 entries, CUST_ID to L6_CHANNEL_TXN_DOUTTA_AVGCNT
dtypes: float64(399), int64(196), object(32)
memory usage: 1.3+ GB
None


In [44]:
train.describe() #number变量，看他们的distribution

,CUST_ID,OPEN_ORG_NUM,bad_good,LAST_OPEN_TENURE_DAYS,G_OS_PRCP_SUM,OS_PRCP_SUM_THREE,OS_PRCP_SUM_SIX,guozhai_flag,L3_DR_AMT,L3_DR_CNT,...,L3_CHANNEL_TXN_DTAIN_AVGCNT,L3_CHANNEL_TXN_DOUTTA_AVGCNT,L6_CHANNEL_TXN_STAIN_AVGAMT,L6_CHANNEL_TXN_SOUTTA_AVGAMT,L6_CHANNEL_TXN_DTAIN_AVGAMT,L6_CHANNEL_TXN_DOUTTA_AVGAMT,L6_CHANNEL_TXN_STAIN_AVGCNT,L6_CHANNEL_TXN_SOUTTA_AVGCNT,L6_CHANNEL_TXN_DTAIN_AVGCNT,L6_CHANNEL_TXN_DOUTTA_AVGCNT
count,2.852850e+05,285285.000000,285285.000000,285285.000000,2.852850e+05,2.852850e+05,2.852850e+05,285285.000000,2.852850e+05,285285.000000,...,285285.0,285285.0,285285.0,285285.0,285285.0,285285.0,285285.0,285285.0,285285.0,285285.0
mean,1.234342e+09,1549.863456,0.017120,2.323087,1.444199e+04,1.422210e+04,7.111052e+03,0.000165,7.754491e+05,3.018084,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,1.000619e+08,1751.476655,0.129718,6.944150,1.928407e+05,1.916681e+05,9.583403e+04,0.012834,8.202532e+06,20.696084,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.000000e+09,101.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,1.160669e+09,304.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,1.249054e+09,918.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,1.309521e+09,2007.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,1.500250e+09,8701.000000,1.000000,88.000000,2.600000e+07,2.666667e+07,1.333333e+07,1.000000,1.348390e+09,3040.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
train.describe(include=['O'])  ##分类变量的distribution

,IDF_TYP_CD,GENDER,CUST_EUP_ACCT_FLAG,CUST_AU_ACCT_FLAG,CUST_SALARY_FINANCIAL_FLAG,CUST_SOCIAL_SECURITYIC_FLAG,CUST_MTFLOW_FLAG,CUST_DOLLER_FLAG,CUST_INTERNATIONAL_GOLD_FLAG,CUST_INTERNATIONAL_COMMON_FLAG,...,FUND_FLAG,C_FUND_FLAG,D_FUND_FLAG,S_FUND_FLAG,BOND_FLAG,CER_BOND_FLAG,BK_BOND_FLAG,CRED_FLAG,RELATED_REPAY_FLAG,TOT_REPAY_FLAG
count,285285,285285,285285,285285,285285,285285,285285,285285,285285,285285,...,285285,285285,285285,285285,285285,285285,285285,285285,285285,285285
unique,18,5,2,2,2,1,1,2,2,2,...,2,3,3,3,2,2,1,2,2,2
top,ZR01,1,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
freq,280187,108746,284545,284875,241310,285285,285285,247866,275442,283128,...,269355,269355,269355,269355,285038,285038,285285,243707,280332,244991


In [48]:
train.describe(include='all')

,CUST_ID,OPEN_ORG_NUM,IDF_TYP_CD,GENDER,bad_good,LAST_OPEN_TENURE_DAYS,G_OS_PRCP_SUM,OS_PRCP_SUM_THREE,OS_PRCP_SUM_SIX,guozhai_flag,...,L3_CHANNEL_TXN_DTAIN_AVGCNT,L3_CHANNEL_TXN_DOUTTA_AVGCNT,L6_CHANNEL_TXN_STAIN_AVGAMT,L6_CHANNEL_TXN_SOUTTA_AVGAMT,L6_CHANNEL_TXN_DTAIN_AVGAMT,L6_CHANNEL_TXN_DOUTTA_AVGAMT,L6_CHANNEL_TXN_STAIN_AVGCNT,L6_CHANNEL_TXN_SOUTTA_AVGCNT,L6_CHANNEL_TXN_DTAIN_AVGCNT,L6_CHANNEL_TXN_DOUTTA_AVGCNT
count,2.852850e+05,285285.000000,285285,285285.0,285285.000000,285285.000000,2.852850e+05,2.852850e+05,2.852850e+05,285285.000000,...,285285.0,285285.0,285285.0,285285.0,285285.0,285285.0,285285.0,285285.0,285285.0,285285.0
unique,NaN,NaN,18,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,ZR01,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,280187,108746.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,1.234342e+09,1549.863456,NaN,NaN,0.017120,2.323087,1.444199e+04,1.422210e+04,7.111052e+03,0.000165,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,1.000619e+08,1751.476655,NaN,NaN,0.129718,6.944150,1.928407e+05,1.916681e+05,9.583403e+04,0.012834,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.000000e+09,101.000000,NaN,NaN,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,1.160669e+09,304.000000,NaN,NaN,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,1.249054e+09,918.000000,NaN,NaN,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,1.309521e+09,2007.000000,NaN,NaN,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
####################################

In [33]:
combine = [train,test]  ###combine一起会变成list格式

In [38]:
train.columns.values  ##所有变量名

array(['CUST_ID', 'OPEN_ORG_NUM', 'IDF_TYP_CD', 'GENDER', 'bad_good',
       'LAST_OPEN_TENURE_DAYS', 'G_OS_PRCP_SUM', 'OS_PRCP_SUM_THREE',
       'OS_PRCP_SUM_SIX', 'guozhai_flag', 'L3_DR_AMT', 'L3_DR_CNT',
       'DR_AMT', 'DR_CNT', 'L3_CR_AMT', 'L3_CR_CNT', 'CR_AMT', 'CR_CNT',
       'CUST_ACCOUNT_CNT', 'CUST_PRODUCT_CNT', 'CUST_SAVING_AMT',
       'CUST_FINA_AMT', 'CUST_PRIFINA_AMT', 'CUST_FOND_AMT',
       'CUST_NADEBT_AMT', 'CUST_YBT_AMT', 'CUST_METAL_AMT', 'CUST_AUM',
       'CUST_EUP_ACCT_FLAG', 'CUST_AU_ACCT_FLAG',
       'CUST_SALARY_FINANCIAL_FLAG', 'CUST_SOCIAL_SECURITYIC_FLAG',
       'CUST_MTFLOW_FLAG', 'CUST_DOLLER_FLAG',
       'CUST_INTERNATIONAL_GOLD_FLAG', 'CUST_INTERNATIONAL_COMMON_FLAG',
       'CUST_INTERNATIONAL_SIL_FLAG', 'CUST_INTERNATIONAL_DIAMOND_FLAG',
       'CUST_GOLD_COMMON_FLAG', 'CUST_STAD_PLATINUM_FLAG',
       'CUST_LUXURY_PLATINUM_FLAG', 'CUST_PLATINUM_FINANCIAL_FLAG',
       'CUST_DIAMOND_FLAG', 'CUST_INFINIT_FLAG', 'CUST_BUSINESS_FLAG',
       'CUS

In [ ]:
#############################

In [129]:
train.loc[[10],['bad_good']]
train.loc[:,['bad_good',"GENDER"]]
train[1:4] #前四行
train.iloc[:,0:1] #取前两列
#loc可以用作取特定行列
#其中rows可以用[1:4]或者“名称”；column是用“名称”或切片

,CUST_ID
0,1000000024
1,1000000515
2,1000003231
3,1000003736
4,1000003852
...,...
285280,1500249699
285281,1500249715
285282,1500249833
285283,1500249910


In [128]:
train.iloc[:,0:1]

,CUST_ID
0,1000000024
1,1000000515
2,1000003231
3,1000003736
4,1000003852
...,...
285280,1500249699
285281,1500249715
285282,1500249833
285283,1500249910


In [90]:
select = train["bad_good"]>0
train[select]
#根据条件筛选特定行
train[train["bad_good"]>0]

,CUST_ID,OPEN_ORG_NUM,IDF_TYP_CD,GENDER,bad_good,LAST_OPEN_TENURE_DAYS,G_OS_PRCP_SUM,OS_PRCP_SUM_THREE,OS_PRCP_SUM_SIX,guozhai_flag,...,L3_CHANNEL_TXN_DTAIN_AVGCNT,L3_CHANNEL_TXN_DOUTTA_AVGCNT,L6_CHANNEL_TXN_STAIN_AVGAMT,L6_CHANNEL_TXN_SOUTTA_AVGAMT,L6_CHANNEL_TXN_DTAIN_AVGAMT,L6_CHANNEL_TXN_DOUTTA_AVGAMT,L6_CHANNEL_TXN_STAIN_AVGCNT,L6_CHANNEL_TXN_SOUTTA_AVGCNT,L6_CHANNEL_TXN_DTAIN_AVGCNT,L6_CHANNEL_TXN_DOUTTA_AVGCNT
2,1000003231,602,ZR01,2,1,6,0.0,466666.66667,233333.33333,0,...,0,0,0,0,0,0,0,0,0,0
12,1000008847,602,ZR01,1,1,28,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,0
13,1000008863,602,ZR01,2,1,0,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,0
39,1000179126,603,ZR01,2,1,0,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,0
93,1000303072,601,ZR01,2,1,0,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273668,1351128273,711,ZR01,2,1,0,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,0
274726,1351639851,137,ZR01,1,1,0,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,0
281467,1500111886,117,ZR01,2,1,0,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,0
283643,1500178346,2205,ZR01,2,1,0,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
np.logical_and(train['bad_good']>0,train['GENDER']==1)
train[np.logical_and(train['bad_good']>0,train['GENDER']==1)]
#多条件共筛选

,CUST_ID,OPEN_ORG_NUM,IDF_TYP_CD,GENDER,bad_good,LAST_OPEN_TENURE_DAYS,G_OS_PRCP_SUM,OS_PRCP_SUM_THREE,OS_PRCP_SUM_SIX,guozhai_flag,...,L3_CHANNEL_TXN_DTAIN_AVGCNT,L3_CHANNEL_TXN_DOUTTA_AVGCNT,L6_CHANNEL_TXN_STAIN_AVGAMT,L6_CHANNEL_TXN_SOUTTA_AVGAMT,L6_CHANNEL_TXN_DTAIN_AVGAMT,L6_CHANNEL_TXN_DOUTTA_AVGAMT,L6_CHANNEL_TXN_STAIN_AVGCNT,L6_CHANNEL_TXN_SOUTTA_AVGCNT,L6_CHANNEL_TXN_DTAIN_AVGCNT,L6_CHANNEL_TXN_DOUTTA_AVGCNT
3072,1010792574,102,ZR01,1,1,0,0.0,0.00000,0.000000,0,...,0,0,0,0,0,0,0,0,0,0
3074,1010793373,102,ZR01,1,1,0,0.0,0.00000,0.000000,0,...,0,0,0,0,0,0,0,0,0,0
3075,1010793397,102,ZR01,1,1,0,0.0,0.00000,0.000000,0,...,0,0,0,0,0,0,0,0,0,0
3076,1010794461,102,ZR01,1,1,0,0.0,0.00000,0.000000,0,...,0,0,0,0,0,0,0,0,0,0
3077,1010794518,102,ZR01,1,1,0,0.0,0.00000,0.000000,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240515,1328993837,1117,ZR01,1,1,0,0.0,0.00000,0.000000,0,...,0,0,0,0,0,0,0,0,0,0
252430,1336934813,1007,ZR01,1,1,0,0.0,133333.33333,66666.666667,0,...,0,0,0,0,0,0,0,0,0,0
265884,1346503760,248,ZR01,1,1,0,0.0,0.00000,0.000000,0,...,0,0,0,0,0,0,0,0,0,0
270190,1349220512,132,ZR01,1,1,0,0.0,0.00000,0.000000,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
train['gender'] = train['GENDER']
train['GENDER'].loc[train['bad_good']>0] = 1
train['GENDER'].loc[train['bad_good']>0]
#对GENDER列生成新的列，或者根据一定条件改变其值

/Users/senlinlidewo/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


2         1
12        1
13        1
39        1
93        1
         ..
273668    1
274726    1
281467    1
283643    1
284125    1
Name: GENDER, Length: 4884, dtype: object

In [ ]:
#quick and dirty code split title from name: http://www.pythonforbeginners.com/dictionary/python-split
    dataset['Title'] = dataset['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]

In [ ]:
#删除行列
#参数说明：
#labels 就是要删除的行列的名字，用列表给定
#axis 默认为0，指删除行，因此删除columns时要指定axis=1；
#index 直接指定要删除的行
#columns 直接指定要删除的列
#inplace=False，默认该删除操作不改变原数据，而是返回一个执行删除操作后的新dataframe；
#inplace=True，则会直接在原数据上进行删除操作，删除后无法返回。

In [113]:
train.drop(train.columns[[20,21]],axis = 1) #列
train.drop(train.index[[1,4]],axis = 0) 
train.drop(train.iloc[:,48:50],axis=1) #删除特定第48-50列
train.ix[:,2:]#删除前两列


,CUST_ID,OPEN_ORG_NUM,IDF_TYP_CD,GENDER,bad_good,LAST_OPEN_TENURE_DAYS,G_OS_PRCP_SUM,OS_PRCP_SUM_THREE,OS_PRCP_SUM_SIX,guozhai_flag,...,L3_CHANNEL_TXN_DOUTTA_AVGCNT,L6_CHANNEL_TXN_STAIN_AVGAMT,L6_CHANNEL_TXN_SOUTTA_AVGAMT,L6_CHANNEL_TXN_DTAIN_AVGAMT,L6_CHANNEL_TXN_DOUTTA_AVGAMT,L6_CHANNEL_TXN_STAIN_AVGCNT,L6_CHANNEL_TXN_SOUTTA_AVGCNT,L6_CHANNEL_TXN_DTAIN_AVGCNT,L6_CHANNEL_TXN_DOUTTA_AVGCNT,gender
0,1000000024,1101,ZR01,2,0,0,204750.0,197750.00000,98875.00000,0,...,0,0,0,0,0,0,0,0,0,2
2,1000003231,602,ZR01,1,1,6,0.0,466666.66667,233333.33333,0,...,0,0,0,0,0,0,0,0,0,1
3,1000003736,602,ZR01,1,0,0,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,1
5,1000003986,602,ZR01,2,0,0,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,2
6,1000004314,602,ZR01,1,0,0,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285280,1500249699,2113,ZR01,1,0,0,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,1
285281,1500249715,5201,ZR01,1,0,0,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,1
285282,1500249833,120,ZR01,1,0,0,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,1
285283,1500249910,3006,ZR01,1,0,0,0.0,0.00000,0.00000,0,...,0,0,0,0,0,0,0,0,0,1


In [161]:
train = train.drop(train.iloc[:,20:50],axis=1)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285285 entries, 0 to 285284
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CUST_ID                285285 non-null  int64  
 1   OPEN_ORG_NUM           285285 non-null  int64  
 2   IDF_TYP_CD             285285 non-null  object 
 3   GENDER                 285285 non-null  object 
 4   bad_good               285285 non-null  int64  
 5   LAST_OPEN_TENURE_DAYS  285285 non-null  int64  
 6   G_OS_PRCP_SUM          285285 non-null  float64
 7   OS_PRCP_SUM_THREE      285285 non-null  float64
 8   OS_PRCP_SUM_SIX        285285 non-null  float64
 9   guozhai_flag           285285 non-null  int64  
 10  L3_DR_AMT              285285 non-null  float64
 11  L3_DR_CNT              285285 non-null  int64  
 12  DR_AMT                 285285 non-null  float64
 13  DR_CNT                 285285 non-null  int64  
 14  L3_CR_AMT              285285 non-nu

In [126]:
train[train["GENDER"]=="1"] #反选

,CUST_ID,OPEN_ORG_NUM,IDF_TYP_CD,GENDER,bad_good,LAST_OPEN_TENURE_DAYS,G_OS_PRCP_SUM,OS_PRCP_SUM_THREE,OS_PRCP_SUM_SIX,guozhai_flag,...,L3_CHANNEL_TXN_DOUTTA_AVGCNT,L6_CHANNEL_TXN_STAIN_AVGAMT,L6_CHANNEL_TXN_SOUTTA_AVGAMT,L6_CHANNEL_TXN_DTAIN_AVGAMT,L6_CHANNEL_TXN_DOUTTA_AVGAMT,L6_CHANNEL_TXN_STAIN_AVGCNT,L6_CHANNEL_TXN_SOUTTA_AVGCNT,L6_CHANNEL_TXN_DTAIN_AVGCNT,L6_CHANNEL_TXN_DOUTTA_AVGCNT,gender
1,1000000515,602,ZR01,1,0,0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,1
3,1000003736,602,ZR01,1,0,0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,1
4,1000003852,602,ZR01,1,0,26,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,1
6,1000004314,602,ZR01,1,0,0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,1
9,1000006667,602,ZR01,1,0,0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285280,1500249699,2113,ZR01,1,0,0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,1
285281,1500249715,5201,ZR01,1,0,0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,1
285282,1500249833,120,ZR01,1,0,0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,1
285283,1500249910,3006,ZR01,1,0,0,0.0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,1


In [160]:
# 文字类列处理
print(train.IDF_TYP_CD.unique())
train.IDF_TYP_CD = train.IDF_TYP_CD.apply(lambda x:x.strip())
train['IDF_TYP_CD'] = [i.replace('ZR',' ') for i in train.IDF_TYP_CD]
#去除某列特定字符。

['01' '20' '03' '09' '22' '21' '04' '07' '11' '16' '06' '05' '08' '02'
 '13' '14' '10' '12']


In [179]:
#数据类型转制
col = list(train.columns)
train[col] = train[col].apply(pd.to_numeric,errors='coerce').fillna(0.0)
train = pd.DataFrame(train,dtype='float')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285285 entries, 0 to 285284
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CUST_ID                285285 non-null  float64
 1   OPEN_ORG_NUM           285285 non-null  float64
 2   IDF_TYP_CD             285285 non-null  float64
 3   GENDER                 285285 non-null  float64
 4   bad_good               285285 non-null  float64
 5   LAST_OPEN_TENURE_DAYS  285285 non-null  float64
 6   G_OS_PRCP_SUM          285285 non-null  float64
 7   OS_PRCP_SUM_THREE      285285 non-null  float64
 8   OS_PRCP_SUM_SIX        285285 non-null  float64
 9   guozhai_flag           285285 non-null  float64
 10  L3_DR_AMT              285285 non-null  float64
 11  L3_DR_CNT              285285 non-null  float64
 12  DR_AMT                 285285 non-null  float64
 13  DR_CNT                 285285 non-null  float64
 14  L3_CR_AMT              285285 non-nu

In [ ]:
##分类变量处理教程
#https://www.kaggle.com/alexisbcook/categorical-variables

In [ ]:
##日期变量处理
import datetime
cols = ['issueDate', 'earliesCreditLine']
for col in cols:
    print(train[col])
for col in cols:
    train[col] = pd.to_datetime(train[col])
    train[col] = train[col].map(lambda x: x.year)
#只提取了年份

In [ ]:
# test train合并后再拆开
    train = data[~data['isDefault'].isnull()].copy()
    target = train_label
    test = data[data['isDefault'].isnull()].copy()

In [ ]:
#第二种方法-根据情况
X_train = train.loc[np.isfinite(train.Survived), :]
X_train = X_train.drop(["Survived"], axis=1).values
y_train = target.values

X_test = train.loc[~np.isfinite(train.Survived), :]

X_test = X_test.drop(["Survived"], axis=1).values

####对列进行便捷操作，apply，mapapply，以及lambda
https://blog.csdn.net/qq_20412595/article/details/81131502